In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
null = None
data = [eval(l) for l in open('datasets/130k_wine_data.json')][0]

In [4]:
data_w_price = [d for d in data if d['price']]

In [6]:
random.shuffle(data_w_price)

In [7]:
train = data_w_price[:80000]
valid = data_w_price[80000:100000]
test  = data_w_price[100000:]

 Data Exploration
 ===